In [ ]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_path = "/home/data/train.json"
with open(train_path, 'r') as f:
    train_data = json.load(f)

print(f"Number of training samples: {len(train_data)}")
print(f"Sample keys in a training record: {list(train_data[0].keys())}")
print("\nSample record:")
print(json.dumps(train_data[0], indent=2))